In [11]:
import random 
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print(device)
# import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import rdkit as rd
from rdkit import Chem
# import torch.nn as nn
import h5py 
from sklearn import metrics 
import numpy as np

In [12]:
def seed_everything(seed: int):
    r"""Sets the seed for generating random numbers in :pytorch:`PyTorch`,
    :obj:`numpy` and Python.

    Args:
        seed (int): The desired seed.
    """
    random.seed(seed)
    np.random.seed(seed)


seed_everything(0)

In [13]:
df = pd.read_csv('./BBBP_extend.csv')

In [14]:
df

,Unnamed: 0,index,A1.0,A1.1,A1.2,A1.3,A1.4,A1.5,A1.6,A1.7,...,E5.123,E5.124,E5.125,E5.126,E5.127,smiles,num,name,p_np,InchiKey
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,1,Propanolol,1,AOLIYNUVSUQXCB-UHFFFAOYSA-N
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,2,Terbutylchlorambucil,1,SZXDOYFHSIIZCF-UHFFFAOYSA-N
2,0,GSDSWSVVBLHKDQ-UHFFFAOYSA-N,-0.314134,0.051664,-0.222201,-0.116477,-0.105048,-0.210600,-0.652453,-0.028389,...,-0.106665,0.170525,-0.347679,0.338022,0.038755,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,3,40730,1,GSDSWSVVBLHKDQ-UHFFFAOYSA-N
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,4,24,1,FAXLXLJWHQJMPK-UHFFFAOYSA-N
4,0,LQOLIRLGBULYKD-JKIFEVAISA-N,-0.468390,0.076436,-0.193967,-0.500219,-0.295834,0.154140,-0.386459,-0.343566,...,-0.227512,0.171720,-0.573027,0.465512,-0.159806,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,5,cloxacillin,1,LQOLIRLGBULYKD-JKIFEVAISA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,0,CHFSOFHQIZKQCR-UHFFFAOYSA-N,-0.369556,0.415470,-0.352184,-0.526113,0.143862,0.095371,-0.535070,-0.011400,...,NaN,NaN,NaN,NaN,NaN,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,2049,licostinel,1,CHFSOFHQIZKQCR-UHFFFAOYSA-N
2035,0,MEFKEPWMEQBLKI-AIRLBKTGSA-N,-0.406700,0.520325,-0.562339,-0.496653,0.152432,-0.153132,-0.885917,-0.067208,...,NaN,NaN,NaN,NaN,NaN,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,2050,ademetionine(adenosyl-methionine),1,MEFKEPWMEQBLKI-AIRLBKTGSA-N
2036,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,2051,mesocarb,1,SAJPRPXALCNNRQ-UHFFFAOYSA-N
2037,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,2052,tofisoline,1,IQLRWLFPGKTMDX-UHFFFAOYSA-N


In [15]:
tox21_data=pd.read_csv("extend_dataset/BBBP/BBBP_train_0over4.csv")

# tox21_data=tox21_data[~tox21_data["A1.0"].isna()]
Tox21_ourimpute_train=tox21_data.iloc[:,2:3202].fillna(0.0).values
Tox21_ourimpute_train_y=tox21_data.loc[:,'p_np'].values

tox21_data_valid=pd.read_csv("extend_dataset/BBBP/BBBP_valid_0over4.csv")

# tox21_data=tox21_data[~tox21_data["A1.0"].isna()]
Tox21_ourimpute_valid=tox21_data_valid.iloc[:,2:3202].fillna(0.0).values
Tox21_ourimpute_valid_y=tox21_data_valid.loc[:,'p_np'].values


tox21_data_test=pd.read_csv("extend_dataset/BBBP/BBBP_test.csv")

# tox21_data=tox21_data[~tox21_data["A1.0"].isna()]
Tox21_ourimpute_test=tox21_data_test.iloc[:,2:3202].fillna(0.0).values
Tox21_ourimpute_test_y=tox21_data_test.loc[:,'p_np'].values

In [16]:
from signaturizer import Signaturizer
# sign = Signaturizer('A1')
import numpy as np
np.bool = np.bool_

In [17]:
impute_layers=["A1","A2","A3","A4","A5","B1","B2","B3","B4","B5",
               "C1","C2","C3","C4","C5","D1","D2","D3","D4","D5",
               "E1","E2","E3","E4","E5"]
data_all=np.empty([df.shape[0],0])
for layer in impute_layers:
    print(layer)
    sign = Signaturizer(layer)
    data_tmp = sign.predict(df["smiles"])
    data_tmp_sig=data_tmp.signature
    data_all=np.concatenate([data_all,data_tmp_sig], axis=1)
    # convert array into dataframe
    DF = pd.DataFrame(data_all, index=df["smiles"])
 
    # save the dataframe as a csv file
    DF.to_csv("extend_dataset/BBBP/SNN_imputed_BBBP_25layers_v2.csv")
#                         Tox21_B2.signature,Tox21_B3.signature,
#                         Tox21_B4.signature,Tox21_B5.signature,
#                        Tox21_C1.signature,Tox21_C2.signature,Tox21_C3.signature,Tox21_C4.signature,Tox21_C5.signature,
#                        Tox21_D1.signature,Tox21_D2.signature,Tox21_D3.signature,Tox21_D4.signature,Tox21_D5.signature,
#                        Tox21_E1.signature,Tox21_E2.signature,Tox21_E3.signature,Tox21_E5.signature], axis=1)


A1


2023-09-29 14:28:28.351464: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38400 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2023-09-29 14:28:28.353227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38400 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


KeyboardInterrupt: 

In [18]:
SNN_impute_data = pd.read_csv("extend_dataset/BBBP/SNN_imputed_BBBP_25layers_v2.csv")

In [19]:
SNN_impute_data
# only_A1_index=~HIV_data["smiles"].isna()

,smiles,0,1,2,3,4,5,6,7,8,...,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,-0.101855,0.044619,0.097480,0.053586,-0.102100,0.099757,0.100755,-0.102482,0.102495,...,0.018432,-0.081596,0.106304,-0.096114,-0.018047,-0.074203,0.113234,-0.089613,-0.081331,0.099192
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,-0.101725,0.030008,0.045964,0.046022,-0.092735,0.098991,-0.101066,0.094391,0.101130,...,-0.080488,0.191656,0.025156,-0.022512,0.160707,0.153611,0.098132,-0.148416,-0.011424,-0.090853
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,0.044534,0.100536,0.100802,0.107339,-0.110283,0.104109,0.011362,-0.012220,0.109306,...,0.080286,-0.127470,0.004751,-0.005866,0.045668,-0.002062,-0.112551,0.126351,-0.023734,-0.061157
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,-0.094195,0.094190,-0.089421,0.093936,-0.094195,-0.094061,-0.094195,-0.094195,-0.094186,...,0.104539,0.117953,-0.006801,-0.071012,0.015149,-0.088104,0.104707,-0.122368,-0.127942,-0.135421
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,-0.099516,0.096248,0.115230,0.100924,-0.065934,-0.072603,0.099869,0.134192,-0.115016,...,-0.119894,0.126521,0.116309,-0.066796,0.047277,0.015528,-0.049841,-0.116483,-0.106030,0.122326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,-0.089638,0.013985,0.091349,0.092633,0.092633,0.092633,0.092404,-0.092619,-0.092633,...,0.129214,-0.065601,-0.086740,0.019164,-0.067376,0.071407,0.157946,0.100020,0.015597,0.037990
2035,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,0.090579,0.090975,0.090974,-0.090976,0.090976,0.077114,0.090976,-0.090976,0.090973,...,0.119832,0.044074,0.114134,-0.019103,0.002314,0.054096,0.145769,-0.122111,0.000578,-0.066024
2036,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,0.082605,-0.112347,-0.010752,-0.033545,-0.033070,0.099392,0.111278,-0.028613,-0.094780,...,0.133034,0.083185,-0.053349,0.119899,0.081890,0.119380,-0.014599,0.020567,-0.103639,-0.146136
2037,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,-0.112296,-0.052799,0.062283,0.079720,-0.012886,0.120334,-0.032120,-0.032748,0.118114,...,0.081327,0.030901,0.047137,0.125875,-0.014538,0.045578,0.048972,0.093004,0.109006,-0.050247


In [48]:
SNN_impute_train_index = pd.read_csv("extend_dataset/BBBP/BBBP_train_index_0over4.csv")
SNN_impute_valid_index = pd.read_csv("extend_dataset/BBBP/BBBP_valid_index_0over4.csv")
SNN_impute_test_index = pd.read_csv("extend_dataset/BBBP/BBBP_test_index.csv")


In [49]:
SNN_impute_train_index=SNN_impute_train_index["0"].values

In [50]:
X_train=SNN_impute_data.iloc[SNN_impute_train_index,1:]

In [51]:

y_train=df["p_np"].values
y_train=y_train[SNN_impute_train_index]

In [54]:
SNN_impute_valid_index

,0
0,2
1,9
2,13
3,14
4,38
...,...
199,1790
200,1796
201,1801
202,1822


In [55]:
SNN_impute_valid_index=SNN_impute_valid_index["0"].values
X_valid=SNN_impute_data.iloc[SNN_impute_valid_index,1:]
y_valid=df["p_np"].values
y_valid=y_valid[SNN_impute_valid_index]



In [56]:
SNN_impute_test_index=SNN_impute_test_index["0"].values
# X=SNN_impute_train_index=SNN_impute_train_index["0"].valuesX[~extend_data["A1.0"].isna()]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [57]:
X_test=SNN_impute_data.iloc[SNN_impute_test_index,1:]
y_test=df["p_np"].values
y_test=y_test[SNN_impute_test_index]

In [58]:
# y=y[~extend_data["A1.0"].isna()]

In [59]:
import numpy as np
from scipy.stats import randint
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn import metrics 
from xgboost import XGBClassifier

names = [
    "Linear SVM",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "XGBoost"
]

classifiers = [
    SVC(kernel="linear", C=0.025),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    AdaBoostClassifier(),
    XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic'),
]

In [60]:

# from sklearn.model_selection import train_test_split
# # i.e. 70 % training dataset and 30 % test datasets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42,stratify=y)
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)
# resample_noimpute_train, resample_noimpute_train_y = rus.fit_resample(X_train, y_train)

# #### resample test set
# rus_test = RandomUnderSampler(random_state=0)
# resample_noimpute_test, resample_noimpute_test_y = rus_test.fit_resample(X_test, y_test)


SNN_impute = []
for name, clf in zip(names, classifiers):
        #ax = plt.subplot(len(datasets), len(classifiers) + 1, i)

        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train, y_train)
        ## validation
        y_valid_pred = clf.predict(X_valid)
        
        
        # performing predictions on the test dataset
        y_test_pred = clf.predict(X_test)
        
        ROC_AUC_valid = metrics.roc_auc_score(y_valid, y_valid_pred, multi_class='ovr')
        ROC_AUC_test = metrics.roc_auc_score(y_test, y_test_pred, multi_class='ovr')
        # metrics are used to find accuracy or error
 
        print()
  
        # using metrics module for accuracy calculation
#         print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
        print("Valid ROC-AUC OF THE MODEL: ",ROC_AUC_valid)
        print("Test ROC-AUC OF THE MODEL: ",ROC_AUC_test)
        no_impute_tmp ={"Method":name,
                         "ROC-AUC":ROC_AUC_test, "Dataset":"Tox21", "Input":"SNN Impute"}
        SNN_impute.append(no_impute_tmp)


Valid ROC-AUC OF THE MODEL:  0.8415546464056107
Test ROC-AUC OF THE MODEL:  0.9807692307692308

Valid ROC-AUC OF THE MODEL:  0.5609000584453536
Test ROC-AUC OF THE MODEL:  0.6810897435897436

Valid ROC-AUC OF THE MODEL:  0.8434248977206311
Test ROC-AUC OF THE MODEL:  0.9455128205128205

Valid ROC-AUC OF THE MODEL:  0.7652834599649329
Test ROC-AUC OF THE MODEL:  0.9110576923076923

Valid ROC-AUC OF THE MODEL:  0.6689070718877849
Test ROC-AUC OF THE MODEL:  0.8373397435897436


In [61]:
snn_df=pd.DataFrame(SNN_impute)
snn_df["ROC-AUC"].mean()

0.8711538461538462

In [62]:

X_train=Tox21_ourimpute_train[:,0:128]
y_train=Tox21_ourimpute_train_y

X_valid=Tox21_ourimpute_valid[:,0:128]
y_valid=Tox21_ourimpute_valid_y

X_test=Tox21_ourimpute_test[:,0:128]
y_test=Tox21_ourimpute_test_y


In [63]:
# from sklearn.model_selection import train_test_split
# # i.e. 70 % training dataset and 30 % test datasets
# X_train, X_test, y_train, y_test = train_test_split(X_SNN_A1, y_SNN_A1, test_size = 0.30, random_state=42,stratify=y_SNN_A1)

In [64]:
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)
# resample_noimpute_train, resample_noimpute_train_y = rus.fit_resample(X_train, y_train)

In [65]:

no_impute = []
for name, clf in zip(names, classifiers):
        #ax = plt.subplot(len(datasets), len(classifiers) + 1, i)

        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train,y_train)
        ## validation
        y_valid_pred = clf.predict(X_valid)
        
        
        # performing predictions on the test dataset
        y_test_pred = clf.predict(X_test)
        
        ROC_AUC_valid = metrics.roc_auc_score(y_valid, y_valid_pred, multi_class='ovr')
        ROC_AUC_test = metrics.roc_auc_score(y_test, y_test_pred, multi_class='ovr')
        # metrics are used to find accuracy or error
 
        print()
  
        # using metrics module for accuracy calculation
#         print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
        print("Valid ROC-AUC OF THE MODEL: ",ROC_AUC_valid)
        print("Test ROC-AUC OF THE MODEL: ",ROC_AUC_test)
        no_impute_tmp ={"Method":name,
                         "ROC-AUC":ROC_AUC_test, "Dataset":"Tox21", "Input":"SNN Impute"}
        no_impute.append(no_impute_tmp)
        
        


Valid ROC-AUC OF THE MODEL:  0.6538461538461539
Test ROC-AUC OF THE MODEL:  0.6378205128205128

Valid ROC-AUC OF THE MODEL:  0.5104166666666667
Test ROC-AUC OF THE MODEL:  0.5488782051282051

Valid ROC-AUC OF THE MODEL:  0.6650641025641026
Test ROC-AUC OF THE MODEL:  0.672275641025641

Valid ROC-AUC OF THE MODEL:  0.578525641025641
Test ROC-AUC OF THE MODEL:  0.6314102564102564

Valid ROC-AUC OF THE MODEL:  0.5568910256410257
Test ROC-AUC OF THE MODEL:  0.5600961538461539


In [66]:
no_impute_df=pd.DataFrame(no_impute)
no_impute_df["ROC-AUC"].mean()

0.6100961538461538